<img style="float: left;" src="http://third.datacastle.cn/pkbigdata/master.other.img/8ef429f9-2032-47b8-bcc4-818fa9e41a25.png" width="50%">

In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
op_train = pd.read_csv('../data/operation_train_new.csv')
tran_train = pd.read_csv('../data/transaction_train_new.csv')
tag_train = pd.read_csv('../data/tag_train_new.csv')

In [3]:
print(op_train.shape)
print(tran_train.shape)

(1460843, 20)
(264654, 27)


### 数据预处理

In [4]:
def operation_len(idx):
    return len(operation_train[operation_train['UID'] == idx])

def transaction_len(idx):
    return len(transaction_train[transaction_train['UID'] == idx])

def find_different_uid(op_uids, tran_uids):
    op_diff_uids = []
    for uid in op_uids:
        if uid not in tran_uids:
            op_diff_uids.append(uid)
        else:
            tran_uids.remove(uid)
#             np.delete(tran_uids, np.where(tran_uids == uid)[0])
    return op_diff_uids, tran_uids

def find_same_uid(src1_uids, src2_uids):
    same_uids = [uid for uid in src1_uids if uid in src2_uids]
    return same_uids

In [5]:
# 处理时间字符串
op_train['time'] = op_train['day'].apply(lambda x: "2018-08-%02d" % x) + ' ' + op_train['time']
op_train['timestamp'] = op_train['time'].apply(lambda x:time.mktime(time.strptime(x,'%Y-%m-%d %H:%M:%S')))

tran_train['time'] = tran_train['day'].apply(lambda x: "2018-08-%02d" % x) + ' ' + tran_train['time']
tran_train['timestamp'] = tran_train['time'].apply(lambda x:time.mktime(time.strptime(x,'%Y-%m-%d %H:%M:%S')))

In [6]:
op_train.head()

,UID,day,mode,success,time,os,version,device1,device2,device_code1,...,device_code3,mac1,mac2,ip1,ip2,wifi,geo_code,ip1_sub,ip2_sub,timestamp
0,10035,30,c8741ce15ceac2a4,1.0,2018-08-30 17:51:50,102,7.0.9,49dd36968dbfadda,OPPO R11,ecb58082e0e9b8e2,...,NaN,NaN,a8dc52f65085212e,55dd8936655c86f6,NaN,NaN,wskx,e58e48fb9215116e,NaN,1.535623e+09
1,16264,16,20a91b45ef8f8221,1.0,2018-08-16 08:36:00,200,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,fc7fc47d6c93f554,NaN,NaN,NaN,3502c553ea2ac187,1.534380e+09
2,13162,8,b668e42707ee9c7b,0.0,2018-08-08 18:09:57,102,7.0.5,630a1adff2a87007,MI MAX 2,1da225cb679a37eb,...,NaN,a8dc52f65085212e,NaN,2147d925e7a8ba3c,NaN,NaN,NaN,3591678eca3f7a23,NaN,1.533723e+09
3,21392,23,c8741ce15ceac2a4,1.0,2018-08-23 23:53:49,102,7.0.9,630a1adff2a87007,MI 5X,d0c285513b785609,...,NaN,NaN,04a2d6c090518faa,c52efc665f0af581,NaN,4ff1553b1c96360e,wm4v,fdb5e20f6a6f2cd7,NaN,1.535040e+09
4,18599,26,acfaded7e04e7ba0,1.0,2018-08-26 11:11:15,102,7.0.0,8399be6b80847240,NaN,27efd832595503f6,...,NaN,NaN,1ab9dc3eae102aa6,0fe293bea342665a,NaN,NaN,NaN,f5009e2c6c13c80c,NaN,1.535253e+09


In [7]:
tran_train.head()

,UID,channel,day,time,trans_amt,amt_src1,merchant,code1,code2,trans_type1,...,bal,amt_src2,acc_id2,acc_id3,geo_code,trans_type2,market_code,market_type,ip1_sub,timestamp
0,19092,102,30,2018-08-30 11:48:47,1459,acdbdb842ac20f1e,abc6bc660561e0eb,NaN,NaN,26bcf43a19df14c8,...,1007,9fefed0a981dcb7a,NaN,NaN,wsbq,102.0,2ecf94369847c748,1.0,45f4ca1f4e9b9cb7,1.535601e+09
1,13465,140,23,2018-08-23 16:58:03,725,4d7831c6f695ab19,7a87b1aa20972d48,NaN,NaN,c2f2023d279665b2,...,100,NaN,NaN,NaN,wte7,105.0,NaN,NaN,2b416f1d7e89ad3b,1.535015e+09
2,13713,140,22,2018-08-22 09:56:39,127,c5fc631370cabc0d,cfd035fc7e0a53b2,NaN,NaN,c2f2023d279665b2,...,100,NaN,NaN,NaN,ws7n,105.0,NaN,NaN,ff71a556d86c4b8f,1.534903e+09
3,22703,140,1,2018-08-01 18:11:45,99317,a571c7fda8b7df37,c71c876b8979028e,NaN,NaN,c2f2023d279665b2,...,100,fbf6bf3c8927414c,NaN,NaN,wqj6,NaN,NaN,NaN,cf1527a0b4173e8f,1.533118e+09
4,17816,140,22,2018-08-22 13:04:02,3361,c5fc631370cabc0d,6628f5ca013e3582,NaN,NaN,c2f2023d279665b2,...,100,NaN,NaN,NaN,NaN,105.0,NaN,NaN,cbc62c2b1cb7deb9,1.534914e+09


In [12]:
op_train = op_train.sort_values(by='timestamp',ascending=True)
tran_train = tran_train.sort_values(by='timestamp',ascending=True)
tag_train = tag_train.sort_values(by='UID', ascending=True)

op_train_gb = op_train.groupby('UID', as_index=True)
tran_train_gb = tran_train.groupby('UID', as_index=True)
# tag_train_gb = tag_train.groupby('UID', as_index=True)

In [23]:
op_train.head(30)

,UID,day,mode,success,time,os,version,device1,device2,device_code1,...,device_code3,mac1,mac2,ip1,ip2,wifi,geo_code,ip1_sub,ip2_sub,timestamp
764126,61286,1,c8741ce15ceac2a4,1.0,2018-08-01 00:00:09,102,7.0.5,d2cf44cec09806cc,VIVO X20A,76fea6d94b07b9e3,...,NaN,NaN,a8dc52f65085212e,07bd9ac95560baf6,NaN,NaN,wt7j,ae6365e1405d2464,NaN,1.533053e+09
1381221,61286,1,c8741ce15ceac2a4,1.0,2018-08-01 00:00:09,102,7.0.5,d2cf44cec09806cc,VIVO X20A,76fea6d94b07b9e3,...,NaN,NaN,a8dc52f65085212e,07bd9ac95560baf6,NaN,NaN,wt7j,ae6365e1405d2464,NaN,1.533053e+09
1068027,61286,1,c8741ce15ceac2a4,1.0,2018-08-01 00:00:11,102,7.0.5,d2cf44cec09806cc,VIVO X20A,76fea6d94b07b9e3,...,NaN,NaN,a8dc52f65085212e,07bd9ac95560baf6,NaN,NaN,wt7j,ae6365e1405d2464,NaN,1.533053e+09
955930,56180,1,d25caee90b27fa9b,1.0,2018-08-01 00:00:12,102,7.0.5,49dd36968dbfadda,NaN,504a5dcdc84688d7,...,NaN,NaN,a8dc52f65085212e,61d4a197084b3f23,NaN,NaN,tzx5,0b82d46f41ed0434,NaN,1.533053e+09
939994,58933,1,c8741ce15ceac2a4,1.0,2018-08-01 00:00:47,102,7.0.5,09baf2f39bc3dc86,HWI-AL00,e0c64f2170dbe4b4,...,NaN,NaN,a8dc52f65085212e,7eb72866bb2d186f,NaN,NaN,wwgq,4eda650aaa653ba4,NaN,1.533053e+09
1084380,58933,1,c8741ce15ceac2a4,1.0,2018-08-01 00:00:47,102,7.0.5,09baf2f39bc3dc86,HWI-AL00,e0c64f2170dbe4b4,...,NaN,NaN,a8dc52f65085212e,7eb72866bb2d186f,NaN,NaN,wwgq,4eda650aaa653ba4,NaN,1.533053e+09
445809,58933,1,c8741ce15ceac2a4,1.0,2018-08-01 00:00:49,102,7.0.5,09baf2f39bc3dc86,HWI-AL00,e0c64f2170dbe4b4,...,NaN,NaN,a8dc52f65085212e,7eb72866bb2d186f,NaN,NaN,wwgq,4eda650aaa653ba4,NaN,1.533053e+09
561603,65623,1,c8741ce15ceac2a4,1.0,2018-08-01 00:00:50,103,7.0.5,aca4977fbe8741e1,IPHONE 6S PLUS,NaN,...,2044012fc7027ef0,NaN,NaN,9a5e2cd3f42cfa81,NaN,fe96480a1ebb918f,NaN,0aad545c46fe53ad,NaN,1.533053e+09
721200,65623,1,c8741ce15ceac2a4,1.0,2018-08-01 00:00:50,103,7.0.5,aca4977fbe8741e1,IPHONE 6S PLUS,NaN,...,2044012fc7027ef0,NaN,NaN,9a5e2cd3f42cfa81,NaN,fe96480a1ebb918f,NaN,0aad545c46fe53ad,NaN,1.533053e+09
571327,58040,1,c8741ce15ceac2a4,1.0,2018-08-01 00:01:07,102,7.0.5,49dd36968dbfadda,R7PLUSM,d8e33dcded8e575e,...,NaN,NaN,04523ec5f7dad67e,6e1a1b43a55eea10,NaN,df0c5b4147f8b086,vb7f,a00c4bc71aaeb89a,NaN,1.533053e+09


In [27]:
op_train[op_train['UID'] == 64478]

,UID,day,mode,success,time,os,version,device1,device2,device_code1,...,device_code3,mac1,mac2,ip1,ip2,wifi,geo_code,ip1_sub,ip2_sub,timestamp
1023896,64478,1,c8741ce15ceac2a4,0.0,2018-08-01 00:02:20,103,7.0.5,bf872c181a38310a,IPHONE 5S,NaN,...,dc26c18cdc83c46b,NaN,NaN,4e8ef9ed6b991caf,NaN,0762cd17a1b44aa9,NaN,9f345fe809c79f42,NaN,1.533053e+09
1395557,64478,1,c8741ce15ceac2a4,0.0,2018-08-01 00:02:21,103,7.0.5,bf872c181a38310a,IPHONE 5S,NaN,...,dc26c18cdc83c46b,NaN,NaN,4e8ef9ed6b991caf,NaN,0762cd17a1b44aa9,NaN,9f345fe809c79f42,NaN,1.533053e+09
1447578,64478,1,acfaded7e04e7ba0,0.0,2018-08-01 00:02:50,103,7.0.5,bf872c181a38310a,IPHONE 5S,NaN,...,dc26c18cdc83c46b,NaN,NaN,4e8ef9ed6b991caf,NaN,0762cd17a1b44aa9,NaN,9f345fe809c79f42,NaN,1.533053e+09
1091154,64478,1,acfaded7e04e7ba0,0.0,2018-08-01 08:25:51,103,7.0.5,bf872c181a38310a,IPHONE 5S,NaN,...,dc26c18cdc83c46b,NaN,NaN,70b9c392965aa800,NaN,0762cd17a1b44aa9,NaN,fee55838bd125afe,NaN,1.533083e+09
822249,64478,1,acfaded7e04e7ba0,1.0,2018-08-01 08:26:08,103,7.0.5,bf872c181a38310a,IPHONE 5S,NaN,...,dc26c18cdc83c46b,NaN,NaN,70b9c392965aa800,NaN,0762cd17a1b44aa9,NaN,fee55838bd125afe,NaN,1.533083e+09
1456703,64478,1,d25caee90b27fa9b,1.0,2018-08-01 08:26:47,103,7.0.5,bf872c181a38310a,NaN,NaN,...,dc26c18cdc83c46b,NaN,NaN,70b9c392965aa800,NaN,NaN,NaN,fee55838bd125afe,NaN,1.533083e+09
1140678,64478,8,c8741ce15ceac2a4,1.0,2018-08-08 08:06:55,103,7.0.5,bf872c181a38310a,IPHONE 5S,NaN,...,dc26c18cdc83c46b,NaN,NaN,51234d60cf795270,NaN,323217c561c661cd,NaN,8b4cd202cab0758a,NaN,1.533687e+09
831033,64478,8,c8741ce15ceac2a4,1.0,2018-08-08 08:06:55,103,7.0.5,bf872c181a38310a,IPHONE 5S,NaN,...,dc26c18cdc83c46b,NaN,NaN,51234d60cf795270,NaN,323217c561c661cd,NaN,8b4cd202cab0758a,NaN,1.533687e+09
907974,64478,8,d25caee90b27fa9b,0.0,2018-08-08 08:07:26,103,7.0.5,bf872c181a38310a,NaN,NaN,...,dc26c18cdc83c46b,NaN,NaN,51234d60cf795270,NaN,NaN,NaN,8b4cd202cab0758a,NaN,1.533687e+09
509207,64478,8,d25caee90b27fa9b,1.0,2018-08-08 08:07:30,103,7.0.5,bf872c181a38310a,NaN,NaN,...,dc26c18cdc83c46b,NaN,NaN,51234d60cf795270,NaN,NaN,NaN,8b4cd202cab0758a,NaN,1.533687e+09


In [26]:
tran_train[tran_train['UID'] == 64478]

,UID,channel,day,time,trans_amt,amt_src1,merchant,code1,code2,trans_type1,...,bal,amt_src2,acc_id2,acc_id3,geo_code,trans_type2,market_code,market_type,ip1_sub,timestamp
213695,64478,140,1,2018-08-01 08:26:40,2818,992d3ce08a4ca702,a56070dcd8517fc6,NaN,NaN,c2f2023d279665b2,...,100,a2aa73cdb6621133,NaN,NaN,NaN,105.0,6cf3afe424fe055e,2.0,fee55838bd125afe,1.533083e+09
191351,64478,140,1,2018-08-01 08:26:40,2818,4d7831c6f695ab19,a56070dcd8517fc6,NaN,NaN,c2f2023d279665b2,...,100,NaN,NaN,NaN,NaN,105.0,NaN,NaN,fee55838bd125afe,1.533083e+09
168696,64478,140,8,2018-08-08 08:07:16,2818,4d7831c6f695ab19,a56070dcd8517fc6,NaN,NaN,c2f2023d279665b2,...,100,NaN,NaN,NaN,NaN,105.0,NaN,NaN,8b4cd202cab0758a,1.533687e+09
262342,64478,140,8,2018-08-08 08:07:16,2818,992d3ce08a4ca702,a56070dcd8517fc6,NaN,NaN,c2f2023d279665b2,...,100,a2aa73cdb6621133,NaN,NaN,NaN,105.0,9406757b48d5f991,2.0,8b4cd202cab0758a,1.533687e+09
193717,64478,140,15,2018-08-15 07:54:53,2818,992d3ce08a4ca702,a56070dcd8517fc6,NaN,NaN,c2f2023d279665b2,...,100,a2aa73cdb6621133,NaN,NaN,NaN,105.0,9406757b48d5f991,2.0,fee55838bd125afe,1.534291e+09
180612,64478,140,15,2018-08-15 07:54:53,2818,4d7831c6f695ab19,a56070dcd8517fc6,NaN,NaN,c2f2023d279665b2,...,100,NaN,NaN,NaN,NaN,105.0,NaN,NaN,fee55838bd125afe,1.534291e+09
218340,64478,140,17,2018-08-17 23:47:21,371,a571c7fda8b7df37,a56070dcd8517fc6,NaN,NaN,c2f2023d279665b2,...,100,6278e75195070f21,NaN,NaN,NaN,105.0,NaN,NaN,f6addb9ed3c8b37d,1.534521e+09
173256,64478,140,17,2018-08-17 23:47:21,5264,992d3ce08a4ca702,a56070dcd8517fc6,NaN,NaN,c2f2023d279665b2,...,100,a2aa73cdb6621133,NaN,NaN,NaN,105.0,dde57bd5e7f3647b,2.0,f6addb9ed3c8b37d,1.534521e+09
241563,64478,140,22,2018-08-22 08:14:10,2818,4d7831c6f695ab19,a56070dcd8517fc6,NaN,NaN,c2f2023d279665b2,...,100,NaN,NaN,NaN,NaN,105.0,NaN,NaN,51ee378378eba085,1.534897e+09
228605,64478,140,22,2018-08-22 08:14:10,2818,992d3ce08a4ca702,a56070dcd8517fc6,NaN,NaN,c2f2023d279665b2,...,100,a2aa73cdb6621133,NaN,NaN,NaN,105.0,9406757b48d5f991,2.0,51ee378378eba085,1.534897e+09


In [24]:
tag_train[tag_train['UID'] == 64478]

,UID,Tag
27900,64478,1


In [ ]:
op_train_uids = [uid for uid, item in op_train.groupby('UID', as_index=False).groups.items()]
tran_train_uids = [uid for uid, item in tran_train.groupby('UID', as_index=False).groups.items()]
tag_train_uids = [uid for uid, item in tag_train.groupby('UID', as_index=False).groups.items()]
print(len(op_train_uids))
print(len(tran_train_uids))
print(len(tag_train_uids))
# op_train_uids = np.array(op_train_uids)
# tran_train_uids = np.array(tran_train_uids)

In [ ]:
op_tag_uids = find_same_uid(op_train_uids, tag_train_uids)
tran_tag_uids = find_same_uid(tran_train_uids, tag_train_uids)
op_tran_tag_uids = find_same_uid(op_train_uids, tran_train_uids)

In [ ]:
print(len(op_tag_uids))
print(len(tran_tag_uids))
print(len(op_tran_tag_uids))

In [ ]:
op_train_gb.count()

In [ ]:
tran_train_gb.count()

In [ ]:
op_train_gb.head()

In [ ]:
tag_true = tag_train[tag_train['Tag']==1]
tag_false = tag_train[tag_train['Tag']==0]

In [ ]:
len_op_true=[]
len_tran_true=[]
len_op_false=[]
len_tran_false=[]
%timeit
print('start tag_true...')
for idx in tag_true['UID'].values:
    len_op_true.append(operation_len(idx))
    len_tran_true.append(transaction_len(idx))
print('start tag_false...')
for idx in tag_false['UID'].values:
    len_op_false.append(operation_len(idx))
    len_tran_false.append(transaction_len(idx))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
len_op_true = np.array(len_op_true)
len_tran_true = np.array(len_tran_true)
len_op_false = np.array(len_op_false)
len_tran_false = np.array(len_tran_false)

In [ ]:
print(len_op_true.shape, len_tran_true.shape)
print(len_op_false.shape, len_tran_false.shape)

In [ ]:
plt.plot(len_op_true)

In [ ]:
plt.plot(len_tran_true)

In [ ]:
plt.plot(len_op_false)

In [ ]:
plt.plot(len_tran_false)